In [1]:
import psycopg2
import pandas as pd
import numpy as np

In [2]:
def get_db_cursor():
    username = "doadmin"
    password = 'yzmodwh2oh16iks6'
    host = 'db-postgresql-cl1-do-user-2276924-0.db.ondigitalocean.com'
    port = 25060
    database = 'MandatoryMetadata'
    schema = 'historical'

    conn = psycopg2.connect(host=host, database=database,
                            user=username, password=password, port=port)
    cur = conn.cursor()
    cur.execute("SET search_path TO " + schema)

    return conn, cur

In [3]:
conn, cur = get_db_cursor()


In [4]:
v1_samples = pd.read_csv('outputFiles/spectra.csv',usecols=[0])

In [5]:
v1_samples

,Unnamed: 0
0,AGD207-408SA0706
1,AGD207-431SA0001
2,AGD207-431SA0002
3,AGD207-431SA0003
4,AGD207-431SA0004
...,...
87362,CW017SA7164
87363,CW017SA7167
87364,CW017SA7168
87365,CW104SA0052


In [6]:
v2_samples = pd.read_sql('select sample_code from MandatoryMetadata', con = conn)

C:\Users\Tsuma Thomas\AppData\Local\Temp\ipykernel_10280\2923516565.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  v2_samples = pd.read_sql('select sample_code from MandatoryMetadata', con = conn)


In [7]:
v2_samples

,sample_code
0,ML-SS21-2
1,ML-SS21-2
2,ML-SS21-2
3,ML-SS21-2
4,ML-SS21-2
...,...
185415,CA053SA1584
185416,CM754SA0017
185417,CA053SA1597
185418,CE445SA0001


In [8]:
v2_samples = v2_samples.loc[~v2_samples['sample_code'].isin(v1_samples['Unnamed: 0'])]

In [9]:
v2_samples

,sample_code
0,ML-SS21-2
1,ML-SS21-2
2,ML-SS21-2
3,ML-SS21-2
4,ML-SS21-2
...,...
185415,CA053SA1584
185416,CM754SA0017
185417,CA053SA1597
185418,CE445SA0001


In [10]:
sample_codes = v2_samples['sample_code'].values

In [11]:
if(len(sample_codes) < 5000):
    count  = len(sample_codes)
    step=count
elif(len(sample_codes) < 70000):
    count = len(sample_codes)
    step=5000
else:
    count = 70000
    step=5000
start = 0


In [12]:
count

70000

In [13]:
start

0

In [14]:
step

5000

In [15]:
spectra = pd.DataFrame()

In [16]:
def convertSpectra(df):
    print("Converting spectra")
    df_ = pd.DataFrame([i[[i for i in i.keys()][0]] for i in df['averaged_spectra'].values],columns = np.arange(522,3977,2))
    df_.index = df.index
    print("Spectra converted")
    return df_

In [ ]:
for i in np.arange(start, count, step):
        
    print("Fetching spectra from {}".format(start))
    samples = [i for i in sample_codes][start:start+step]
    print(len(samples))
    query = f"SELECT spectraldata.metadata_id, averaged_spectra, mandatorymetadata.sample_code  FROM spectraldata INNER JOIN mandatorymetadata ON mandatorymetadata.metadata_id = spectraldata.metadata_id WHERE is_finalized=True AND passed=True AND is_active=True AND averaged=True AND sample_code IN {str(samples).replace('[','(').replace(']',')')}"

    _ = pd.read_sql(query, con=conn)
    print("This is the spectra",_)
    spectra = pd.concat([spectra, _], axis=0)
    start = start + step
    if (count-step) > 5000:
        step=5000
    else:
        step = count-step

conn.close()
spectra = spectra[['sample_code', 'averaged_spectra']]
spectra = spectra.set_index('sample_code')
spectra = convertSpectra(spectra)


spectra.to_csv("./outputFiles/spectraldata.csv")

return spectra

Fetching spectra from 0
5000


C:\Users\Tsuma Thomas\AppData\Local\Temp\ipykernel_10280\3978604781.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
